In [ ]:
# this involves standardization and normalization and is important as some models are sensitive to the feature scales

'''
for the small datasets the numbers with very high fluctuations can distort the model so better to do scaling
standard is used when the datas are normally distibuted and bell shaped
minmax is rare and for some specific requirements only
'''
import pandas as pd
from sklearn.preprocessing import StandardScaler ,MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score,KFold
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import Ridge, RidgeCV, LinearRegression # this beacause our dataset is smaller and it helps prevent overfitting
from sklearn.compose import ColumnTransformer # using this to broaden my knowledge


In [ ]:
df= pd.read_csv("cleaned_data.csv")

# separate target and features as scaling is done in features
x=df[['TV','Radio','Newspaper']] # for selecting more columns we have to use double squared
y=df['Sales']

# for feature scaling we have to split the datasets into train and test to prevent the data leakage and donot do cheating
x_train, x_test, y_train, y_test= train_test_split(x,y, test_size=0.3, random_state=42)


In [ ]:
# standard scaler using columntransformer

# this is a object and blueprint of what actually has to be done (no caluculations)
scaler = ColumnTransformer(
    transformers=[ 
        # tuple used as we might apply different methods i.e. scaling or encoding according to the columns
        ('scaled', StandardScaler(),['TV','Radio','Newspaper'])

    ],
    remainder='passthrough' # this keeps the unused columns as it is without removing
)

# now scaling train and test data which will be numpy array

# fit transform done so scaler lerans about the statisitcs of training data and tranforms / scales them
x_train_Scaled= scaler.fit_transform(x_train) 

# only transforming test data as fiting will leak the datas because we are predicting the test data and doing fiting will help model to see data previously.
x_test_Scaled= scaler.transform(x_test) 

# converting these arrays to the dataframe i.e. creates new dataframe
x_train_Scaled = pd.DataFrame(x_train_Scaled,columns=['TV','Radio','Newspaper'])
x_test_Scaled=pd.DataFrame(x_test_Scaled,columns=['TV','Radio','Newspaper'])





In [ ]:
# apply minmax scaler to compare
minmax=MinMaxScaler()

x_train_mm=minmax.fit_transform(x_train)
x_test_mm = minmax.transform(x_test)

# converting the numpy arrays from fit transform to datframe
x_train_df = pd.DataFrame(x_train_mm, columns=x.columns) 
# this x.columns sues the all columns original of the splited x feature i.e. the easiest way to do



In [ ]:
# now comparing the two scaling models
print ('standard scaler summary')
print(x_test_Scaled.describe())

print('\n')

print(' summary statistics minmax')
print(x_train_df.describe())



In [ ]:
# modelling starts from here
# using linear regression
import numpy as np
model=LinearRegression()
'''
model.fit(x_train_Scaled, y_train) # this learns from the trained data

y_pred=model.predict(x_test_Scaled)
rmse=np.sqrt(mean_squared_error(y_test, y_pred))

print(" Lr without cross validation")
print("rmse: ", rmse)
'''

# linear regression with cross validation
''' 
cross valdiation expects a score where higher is better and it actually expects the higher values rather than the smaller 
so when we use mse, it is error and we need it to be smaller not large as cv always search for larger
so if we use neagtive mse then it will pick larger which iis the less mse thus best model
'''
corssV=cross_val_score(model, x_train_Scaled,y_train,cv=5,scoring='neg_mean_squared_error' )
rmse= np.sqrt(-corssV)
r2= cross_val_score(model, x_train_Scaled, y_train,scoring='r2',cv=5)
print('r2 score',r2)
print("rmse score:",rmse)

In [ ]:
# now analysing the best fold and refiting the model using that

best= np.argmin(rmse)   # this np.argmin returns the index of the smallest value in np array
print("best fold is of: ", best) # we got the index of best
'''
cv gives us the values of the best models and helps us identify the different sets
but if we want to refit our model with this best dataset we have to manusally refit it
cv uses kfold under the hood i.e. kfold is the default for splitting

'''

refit= KFold(n_splits=5, shuffle=True, random_state=42)
fd = list(refit.split(x_train_Scaled))  # this gives the list of the indexes of the splited folds ie.e list of all 5 folds

# here happens the actual spliting of the dtaa to the best form
trainIndex, valIndex =fd[best]  # this gives us the actual index of the best fold which is situated at the list given by kfold

''' 
x_train_Scaled is the dataframe and indexing of dtaaframe is wrong so we have to convert it to the numpy array 1st and then only
indexing is done
'''
x_train_Scaled_np =x_train_Scaled.values  # coverts to numpy
y_train_np =y_train.values


# this is doing indexing like a numpy array using integer array
xFtrain, xFval=x_train_Scaled_np[trainIndex], x_train_Scaled_np[valIndex] # here splitting occurs

yFtrain, yFval =y_train_np[trainIndex], y_train_np[valIndex]

# noe fiting on this fold for the best model
final=LinearRegression()
final.fit(xFtrain, yFtrain)

# evaluating metrics
ypredf=final.predict(x_test_Scaled)
rmsef= np.sqrt(mean_squared_error(y_test,ypredf))
r2f= r2_score(y_test,ypredf)

print("rmse final model ", rmsef)
print('r2 final model',r2f)

In [ ]:
# visualization of linear regression
import matplotlib.pyplot as plt


plt.figure(figsize=(8,6))
plt.scatter(y_test, ypredf, alpha=0.7) # aplha means transparency 

# this line below gives the points from where the line should be plotted
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')   # this is as (x,y) so x is y_test.min and max and y is also y_test.min and max
plt.xlabel('Actual Sales')
plt.ylabel('Predicted Sales')
plt.title(' Actual vs Predicted')
plt.show()



In [ ]:
# reisual plot to identify the outliers and errors

residuals = y_test - ypredf
import matplotlib.pyplot as plt
plt.figure(figsize=(7,5))
plt.scatter(ypredf, residuals, alpha=0.7)
plt.hlines(0, ypredf.min(), ypredf.max(), colors='r', linestyles='dashed')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals (Actual - Predicted)')
plt.title('Residual Plot for Lienar Regression')
plt.show()

''' 
 this plot showed some outliers and errors in the model which might afftect the vairance assumptions 
 so we have to standardize those residues mannually
'''

# standardizing the residuals
stdresiduals=residuals / np.std(residuals) 
# this divides each residuals by their sd and if that is >3 then it is outlier as per by rule i.e. >+-3


# now flag potential outliers 
outliers =np.where(np.abs(stdresiduals) > 3)[0]  
'''
this gives the index of the outliers
the above line gives the tuple as (a,b) where a is the index of outlier and b denotes that 3 i.e. the number for which we are checking
then [0] helps us  to get the index of outliers i.e. the first element of the tuple
'''
print("linear regression outliers at index: ", outliers)

''' 
running this i found the outlier at index 34 i.e. from 0 to 34
noe checking the value
this index of outlier is the index of the test data and not the original dataset
'''
index= 34
# Get the *true original index* from the dataset
print("True index from original dataset:", y_test.index[34])

# Get the actual values:
print("Actual value:", y_test.iloc[34])
print("Predicted value:", ypredf[34])
print("Residual:", residuals.iloc[34])
print("Standardized residual:", stdresiduals.iloc[34])


In [ ]:
''' 
as this datseet is small and this one residual outier is high and for the preediction of the sales, it might cause error 
so we have to use more robust model like huber i.e. less sensitive to outliers
it squares the less residues
it gives the absolute value of the large residues
it identifies and separates the outliers and normal values using epsilon value
if e= 1.35 and is the absolute error is less than e then it is normal and huber squares them
otherwise it treates as the outlier and use the absolute value

'''
from sklearn.linear_model import HuberRegressor

huber= HuberRegressor()
huber.fit(x_train, y_train)
ypredhuber=huber.predict(x_test)
rmsehuber=np.sqrt(mean_squared_error(y_test, ypredhuber))
r2huber=r2_score(y_test, ypredhuber)
print("rmse huber: ", rmsehuber)
print("r2 huber: ", r2huber)


In [ ]:
df =pd.read_csv("cleaned_data.csv")

x=df[['TV','Radio']]
y=df['Sales']



In [ ]:
# train test
x_train, x_test, y_train, y_test=train_test_split(x,y, test_size=0.3, random_state=42)

# scaling these features
scaler=StandardScaler()
x_scaled_train=scaler.fit_transform(x_train)
x_test_scaled= scaler.transform(x_test)


In [ ]:
# using ridge regresion
'''  
alpha is the parameter that gives how much value to shrink. 
high value = underfiting more regularization, coefficients becomes smaller
low = overfiting
we have to tune this alpha value as it might result to the wrong value and affect our estimation
to do this we can use the ridgecv i.e. the alpha tuner for ridge regression

'''
import numpy as np
alpha=np.logspace(-4,4,50) # this cretaes a  numpy array of 50 alphas from 10 pwer -4 to 4
ridgecv = RidgeCV(alphas=alpha, cv=5,scoring='neg_mean_squared_error')

# this line below selects the best alpha with the best average cv score and then fits the whole model on that alpha
ridgecv.fit(x_scaled_train, y_train)  # this fits the model of data

ypred=ridgecv.predict(x_test_scaled)  

coefDF= pd.DataFrame({'Feature': x.columns, 'coefficient':ridgecv.coef_})
# evaluation of model
mse=mean_squared_error(y_test,ypred)

rmse=np.sqrt(mse)

r2=r2_score(y_test,ypred)

print("best alpha:", ridgecv.alpha_)
print("mse:", mse)
print("rmse:", rmse)
print("r2 score:", r2)





In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(7,6))
plt.scatter(y_test, ypred, alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')  # diagonal line for perfect prediction
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Ridge Regression: Actual vs Predicted')
plt.show()


In [ ]:
residuals = y_test - ypred

plt.figure(figsize=(7,5))
plt.scatter(ypred, residuals, alpha=0.7)
plt.hlines(0, ypred.min(), ypred.max(), colors='r', linestyles='dashed')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals (Actual - Predicted)')
plt.title('Residual Plot for Ridge Regression')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
print("Length of x.columns:", len(x.columns))
print("x.columns:", list(x.columns))

print("Shape of final.coef_:", final.coef_.shape)
print("final.coef_:", final.coef_)

print("Shape of ridgecv.coef_:", ridgecv.coef_.shape)
print("ridgecv.coef_:", ridgecv.coef_)

print("Length of Model list Linear Regression:", len(['Linear Regression'] * len(x.columns)))
print("Length of Model list Ridge Regression:", len(['Ridge Regression'] * len(x.columns)))


In [ ]:
# bar plot of the coefficients

# coefficients of the linear regression
''' 
when we make the df from pandas from a dictionary, every list or column must be of the same length
as features and coefficients are of length 3 but the model is of only 1 length string
so either we have to drop this or we have to make this lenth 3
so we did make this length 3
here the x.columns has 2 length only as the x has 2 columns as tv and radio 
so we have to make this plot list with 2 models only

'''
print(final.coef_.shape)
coeflr = pd.DataFrame({
    'Feature': x.columns,
    'coefficient': final.coef_[:len(x.columns)],        # shape (3,) matches x.columns length
    'Model': ['Linear Regression'] * len(x.columns)
})

# coefficients of the ridge  regression
coefridge = pd.DataFrame({
    'Feature': x.columns,
    'coefficient': ridgecv.coef_,
    'Model': ['Ridge Regression'] * len(x.columns)
})
# combining both for plot
coefall= pd.concat([coeflr, coefridge], ignore_index=True)

# plotting the coefficients
plt.figure(figsize=(10,8))

# hue splits the data with the model type and for each model coefficients will be sideways and color will be asssigned different
sns.barplot(data=coefall, x='Feature', y='coefficient', hue='Model' , palette='viridis')

plt.title('comparison between coefficients of Linear and Ridge Regression')
plt.ylabel('coefficient')
plt.show()

'''
 this plot shows the coefficient of the varous features as caluclated by the linear i.e. non penalized and penalized i.e. ridge regression
The ridge regression coefficients are smaller than the linear regression coefficients, indicating that ridge regression has applied regularization to reduce the magnitude of the coefficients, which helps prevent overfitting.
'''